In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
%cd /content/drive/MyDrive/stock_predict

/content/drive/MyDrive/stock_predict


In [22]:
import pandas as pd
df = pd.read_excel('SamsungResult_20220103-20230102.xlsx') #2022 1월~ 2023 1월까지의 뉴스

df = df[['일자', '제목', '키워드',
       '특성추출(가중치순 상위 50개)', '본문']]

samsung_stock_prices = pd.read_csv('samsung_2022-01-01~2023-02-02.csv') # 22년도 1월~ 23년도 2월까지의 삼성전자 주식가격

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [23]:
samsung_stock_prices['날짜']= pd.to_datetime(samsung_stock_prices['날짜'])
samsung_stock_prices['날짜_하루뒤']= samsung_stock_prices['날짜']+ pd.DateOffset(days=1)
samsung_stock_prices['날짜_일주일뒤']=samsung_stock_prices['날짜']+pd.DateOffset(days=7) #일주일 뒤의 변동률을 계산할 수도 있을까봐 만들었습니다.

samsung_stock_prices

,날짜,종가,시가,고가,저가,거래량,변동 %,날짜_하루뒤,날짜_일주일뒤
0,2023-02-02,"63,500","63,200","63,900","62,600",23.12M,2.75%,2023-02-03,2023-02-09
1,2023-02-01,"61,800","62,500","62,600","61,000",0.11K,1.31%,2023-02-02,2023-02-08
2,2023-01-31,"61,000","63,400","63,700","61,000",1.28K,-3.63%,2023-02-01,2023-02-07
3,2023-01-30,"63,300","64,900","64,900","63,100",19.15M,-2.01%,2023-01-31,2023-02-06
4,2023-01-29,"64,600","64,600","64,600","64,600",NaN,0.00%,2023-01-30,2023-02-05
...,...,...,...,...,...,...,...,...,...
320,2022-01-07,"78,300","78,100","78,400","77,400",15.10M,1.82%,2022-01-08,2022-01-14
321,2022-01-06,"76,900","76,700","77,600","76,700",8.25K,-0.65%,2022-01-07,2022-01-13
322,2022-01-05,"77,400","78,800","79,000","76,400",1.59K,-1.65%,2022-01-06,2022-01-12
323,2022-01-04,"78,700","78,800","79,200","78,300",3.28K,0.13%,2022-01-05,2022-01-11


In [24]:
# '종가' 컬럼 데이터에서 쉼표(,) 제거하고 숫자로 변환
samsung_stock_prices['종가'] = samsung_stock_prices['종가'].str.replace(',', '').astype(float)

# '어제 종가' 컬럼 추가
#samsung_stock_prices['어제 종가'] = samsung_stock_prices['종가'].shift(-1)
samsung_stock_prices['내일 종가']= samsung_stock_prices['종가'].shift(1)
samsung_stock_prices

,날짜,종가,시가,고가,저가,거래량,변동 %,날짜_하루뒤,날짜_일주일뒤,내일 종가
0,2023-02-02,63500.0,"63,200","63,900","62,600",23.12M,2.75%,2023-02-03,2023-02-09,NaN
1,2023-02-01,61800.0,"62,500","62,600","61,000",0.11K,1.31%,2023-02-02,2023-02-08,63500.0
2,2023-01-31,61000.0,"63,400","63,700","61,000",1.28K,-3.63%,2023-02-01,2023-02-07,61800.0
3,2023-01-30,63300.0,"64,900","64,900","63,100",19.15M,-2.01%,2023-01-31,2023-02-06,61000.0
4,2023-01-29,64600.0,"64,600","64,600","64,600",NaN,0.00%,2023-01-30,2023-02-05,63300.0
...,...,...,...,...,...,...,...,...,...,...
320,2022-01-07,78300.0,"78,100","78,400","77,400",15.10M,1.82%,2022-01-08,2022-01-14,78300.0
321,2022-01-06,76900.0,"76,700","77,600","76,700",8.25K,-0.65%,2022-01-07,2022-01-13,78300.0
322,2022-01-05,77400.0,"78,800","79,000","76,400",1.59K,-1.65%,2022-01-06,2022-01-12,76900.0
323,2022-01-04,78700.0,"78,800","79,200","78,300",3.28K,0.13%,2022-01-05,2022-01-11,77400.0


In [25]:
# '하루변동률' 계산  '오늘'의 기사가 '내일'의 주가에 미친 영향을 보기 위해
samsung_stock_prices['하루변동률'] = (samsung_stock_prices['내일 종가'] - samsung_stock_prices['종가']) / samsung_stock_prices['종가'] *100
samsung_stock_prices


,날짜,종가,시가,고가,저가,거래량,변동 %,날짜_하루뒤,날짜_일주일뒤,내일 종가,하루변동률
0,2023-02-02,63500.0,"63,200","63,900","62,600",23.12M,2.75%,2023-02-03,2023-02-09,NaN,NaN
1,2023-02-01,61800.0,"62,500","62,600","61,000",0.11K,1.31%,2023-02-02,2023-02-08,63500.0,2.750809
2,2023-01-31,61000.0,"63,400","63,700","61,000",1.28K,-3.63%,2023-02-01,2023-02-07,61800.0,1.311475
3,2023-01-30,63300.0,"64,900","64,900","63,100",19.15M,-2.01%,2023-01-31,2023-02-06,61000.0,-3.633491
4,2023-01-29,64600.0,"64,600","64,600","64,600",NaN,0.00%,2023-01-30,2023-02-05,63300.0,-2.012384
...,...,...,...,...,...,...,...,...,...,...,...
320,2022-01-07,78300.0,"78,100","78,400","77,400",15.10M,1.82%,2022-01-08,2022-01-14,78300.0,0.000000
321,2022-01-06,76900.0,"76,700","77,600","76,700",8.25K,-0.65%,2022-01-07,2022-01-13,78300.0,1.820546
322,2022-01-05,77400.0,"78,800","79,000","76,400",1.59K,-1.65%,2022-01-06,2022-01-12,76900.0,-0.645995
323,2022-01-04,78700.0,"78,800","79,200","78,300",3.28K,0.13%,2022-01-05,2022-01-11,77400.0,-1.651842


In [26]:
# '일자' 열의 날짜 형식 변경 ('20230102' -> '2023-01-02')
df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
# '일자' 열의 날짜 형식 변경 ('2023-01-02' 형식으로)
df['일자'] = pd.to_datetime(df['일자']).dt.strftime('%Y-%m-%d')

# '날짜' 열의 날짜 형식 변경 ('2023-01-02' 형식으로)
samsung_stock_prices['날짜'] = pd.to_datetime(samsung_stock_prices['날짜']).dt.strftime('%Y-%m-%d')

In [27]:
merged_df = pd.merge(df,samsung_stock_prices,how='left',left_on='일자',right_on='날짜') #날짜를 기준으로 기사 파일과  주가 파일을 병합
df['변동률'] = merged_df['하루변동률'] #병합 후 '하루 변동률' 컬럼만 붙이기
df

,일자,제목,키워드,특성추출(가중치순 상위 50개),본문,변동률
0,2023-01-02,삼성전자 선제 투자 기술 차별화로 '반도체 초강대국' 주도한다,"투자,삼성전자,선제,차별화,반도체,주도,삼선전자,DX,반도체,사업,경계,통합,시너지...","삼성전자,반도체,차세대,파운드리,미국,ai,삼성,경쟁력,3나노","삼선전자의 DX(반도체) 부문은 사업 간 경계를 뛰어넘는 통합 시너지를 확대하고, ...",-0.180180
1,2023-01-02,“삼성전자 SK하이닉스 주가 올해는 괜찮다” 믿어도 될까 [이종화의 세돌아이],"주가,삼성전자,SK,하이닉스,이종화,세돌아이,삼성전자,삼성전자,29%,,하이닉스,4...","반도체,삼성전자,하이닉스,유진투자증권,추경호,세액공제율,이번주,지원책,세미컨덕터,인...","작년 삼성전자 -29%, 하이닉스 -41% \n주가 폭락 이듬해엔 급등한 사례 많아...",-0.180180
2,2023-01-02,삼성전자 시무식 개최...한종희 “다시 한번 한계의 벽을 넘자”,"개최,삼성전자,시무식,한종희,한계,경영진,임직원,참석,삼성전자,수원,삼성,디지털,시...","부회장,시무식,경쟁력,삼성,임직원,삼성전자,경계현,한종희,신년사,제공해,사업장,수원...","주요 경영진, 임직원 등 150여명 참석\n\n2일 삼성전자는 오전 수원 ‘삼성 디...",-0.180180
3,2023-01-02,"한종희 삼성전자 부회장 ""다시 한번 한계의 벽 넘자""","부회장,한종희,삼성전자,한계,삼성전자,2일,수원,삼성,디지털,시티,시무식,개최,시무...","부회장,시무식,경쟁력,삼성,임직원,경계현,한종희,부문장,신년사,제공해,사업장,삼성전...",[머니투데이 오문영 기자] \n삼성전자가 2일 오전 수원 삼성 디지털 시티에서 '2...,-0.180180
4,2023-01-02,"삼성전자 한종희 경계현 대표 ""과감한 도전과 변신으로 도약 전환점 만들자""","삼성전자,한종희,경계현,대표,도전,변신,도약,전환점,위기,대응,위상,경쟁력,경영체질...","경쟁력,부회장,삼성전자,임직원,한종희,경계현,삼성,사업장,제공해,차별화,시무식,전환...",현재 위기 대응에 따라 위상과 경쟁력 달라져 \n경영체질과 조직문화 새롭게 변화시켜...,-0.180180
...,...,...,...,...,...,...
5429,2022-01-03,"'꼰대 문화' 버려라, 실패해도 좋아 삼성전자 파격 신년사","꼰대,문화,실패,신년사,삼성전자,파격,과거,비즈니스,모델,전략,경직,프로세스,시대,...","임직원,삼성전자,부회장,사업부장,신년사,시무식,한종희,경계현,노태문,이재승,이정배,...","[머니투데이 심재현 기자] \n""과거의 비즈니스 모델과 전략, 경직된 프로세스와 시...",0.127226
5430,2022-01-03,한종희 경계현 삼성전자 대표 “가치있는 고객경험으로 사업 품격 높이자”,"한종희,경계현,가치,삼성전자,대표,고객경험,사업,품격,기업,신년사,삼성전자,최고,경...","삼성전자,임직원,사업부장,부회장,한종희,경계현,시무식,신년사,노태문,이재승,최시영,...",삼성전자 최고경영자(CEO)들이 가치 있는 고객경험을 통해 사업의 품격을 높여나가자...,0.127226
5431,2022-01-03,"1Q 반도체 가격, 삼성전자 시안 공장에 달렸다","가격,반도체,공장,삼성전자,시안,삼성전자,공장,중국,시안,록다운,환경,반도체,가격,...","반도체,삼성전자,연구원,낸드플래시,삼성증권,자릿수,공급사,실리콘웨이퍼,생산능력,생산...",[머니투데이 정인지 기자] 삼성전자의 중국 시안 공장이 록다운되면서 반도체 가격 환...,0.127226
5432,2022-01-03,"삼성전자, 메모리 초격차 스마트폰 가전 시너지로 매출 300조시대 연다","삼성전자,메모리,초격차,스마트폰,가전,시너지,매출,시대,연다,재계,삼성전자,가격,반...","반도체,삼성전자,다운사이클,차세대,경쟁력,산업은행,스마트폰",◆ 2022 재계가 뛴다 ◆\n 삼성전자는 올해 가격 반등이 전망되는 메모리 반도체...,0.127226


In [28]:
# '변동률' 열의 NaN 값을 0으로 대체.  자료가 짤려있어서 Nan 값이 있음...
df['변동률'].fillna(0, inplace=True)
df

,일자,제목,키워드,특성추출(가중치순 상위 50개),본문,변동률
0,2023-01-02,삼성전자 선제 투자 기술 차별화로 '반도체 초강대국' 주도한다,"투자,삼성전자,선제,차별화,반도체,주도,삼선전자,DX,반도체,사업,경계,통합,시너지...","삼성전자,반도체,차세대,파운드리,미국,ai,삼성,경쟁력,3나노","삼선전자의 DX(반도체) 부문은 사업 간 경계를 뛰어넘는 통합 시너지를 확대하고, ...",-0.180180
1,2023-01-02,“삼성전자 SK하이닉스 주가 올해는 괜찮다” 믿어도 될까 [이종화의 세돌아이],"주가,삼성전자,SK,하이닉스,이종화,세돌아이,삼성전자,삼성전자,29%,,하이닉스,4...","반도체,삼성전자,하이닉스,유진투자증권,추경호,세액공제율,이번주,지원책,세미컨덕터,인...","작년 삼성전자 -29%, 하이닉스 -41% \n주가 폭락 이듬해엔 급등한 사례 많아...",-0.180180
2,2023-01-02,삼성전자 시무식 개최...한종희 “다시 한번 한계의 벽을 넘자”,"개최,삼성전자,시무식,한종희,한계,경영진,임직원,참석,삼성전자,수원,삼성,디지털,시...","부회장,시무식,경쟁력,삼성,임직원,삼성전자,경계현,한종희,신년사,제공해,사업장,수원...","주요 경영진, 임직원 등 150여명 참석\n\n2일 삼성전자는 오전 수원 ‘삼성 디...",-0.180180
3,2023-01-02,"한종희 삼성전자 부회장 ""다시 한번 한계의 벽 넘자""","부회장,한종희,삼성전자,한계,삼성전자,2일,수원,삼성,디지털,시티,시무식,개최,시무...","부회장,시무식,경쟁력,삼성,임직원,경계현,한종희,부문장,신년사,제공해,사업장,삼성전...",[머니투데이 오문영 기자] \n삼성전자가 2일 오전 수원 삼성 디지털 시티에서 '2...,-0.180180
4,2023-01-02,"삼성전자 한종희 경계현 대표 ""과감한 도전과 변신으로 도약 전환점 만들자""","삼성전자,한종희,경계현,대표,도전,변신,도약,전환점,위기,대응,위상,경쟁력,경영체질...","경쟁력,부회장,삼성전자,임직원,한종희,경계현,삼성,사업장,제공해,차별화,시무식,전환...",현재 위기 대응에 따라 위상과 경쟁력 달라져 \n경영체질과 조직문화 새롭게 변화시켜...,-0.180180
...,...,...,...,...,...,...
5429,2022-01-03,"'꼰대 문화' 버려라, 실패해도 좋아 삼성전자 파격 신년사","꼰대,문화,실패,신년사,삼성전자,파격,과거,비즈니스,모델,전략,경직,프로세스,시대,...","임직원,삼성전자,부회장,사업부장,신년사,시무식,한종희,경계현,노태문,이재승,이정배,...","[머니투데이 심재현 기자] \n""과거의 비즈니스 모델과 전략, 경직된 프로세스와 시...",0.127226
5430,2022-01-03,한종희 경계현 삼성전자 대표 “가치있는 고객경험으로 사업 품격 높이자”,"한종희,경계현,가치,삼성전자,대표,고객경험,사업,품격,기업,신년사,삼성전자,최고,경...","삼성전자,임직원,사업부장,부회장,한종희,경계현,시무식,신년사,노태문,이재승,최시영,...",삼성전자 최고경영자(CEO)들이 가치 있는 고객경험을 통해 사업의 품격을 높여나가자...,0.127226
5431,2022-01-03,"1Q 반도체 가격, 삼성전자 시안 공장에 달렸다","가격,반도체,공장,삼성전자,시안,삼성전자,공장,중국,시안,록다운,환경,반도체,가격,...","반도체,삼성전자,연구원,낸드플래시,삼성증권,자릿수,공급사,실리콘웨이퍼,생산능력,생산...",[머니투데이 정인지 기자] 삼성전자의 중국 시안 공장이 록다운되면서 반도체 가격 환...,0.127226
5432,2022-01-03,"삼성전자, 메모리 초격차 스마트폰 가전 시너지로 매출 300조시대 연다","삼성전자,메모리,초격차,스마트폰,가전,시너지,매출,시대,연다,재계,삼성전자,가격,반...","반도체,삼성전자,다운사이클,차세대,경쟁력,산업은행,스마트폰",◆ 2022 재계가 뛴다 ◆\n 삼성전자는 올해 가격 반등이 전망되는 메모리 반도체...,0.127226


In [31]:
texts = list(df[:10]['제목'])
texts

["삼성전자 선제 투자 기술 차별화로  '반도체 초강대국' 주도한다",
 '“삼성전자 SK하이닉스 주가 올해는 괜찮다” 믿어도 될까 [이종화의 세돌아이]',
 '삼성전자 시무식 개최...한종희 “다시 한번 한계의 벽을 넘자”',
 '한종희 삼성전자 부회장 "다시 한번 한계의 벽 넘자"',
 '삼성전자 한종희 경계현 대표 "과감한 도전과 변신으로 도약 전환점 만들자"',
 '\u200b삼성전자, 2023년 시무식 개최 한종희 "新환경경영전략 본격화 원년"',
 '[신년사]한종희 삼성전자 부회장 "다시 한번 한계의 벽을 넘자"',
 '올해 주도株  삼성전자 POSCO홀딩스 한화솔루션',
 '삼성전자 셀트리온 "두 번 실망은 없다"',
 '[데이터로 보는 증시]삼성전자 KT&G, 기관 외국인 코스피 주간 순매수 1위(12월 26일~29일)']

In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

tf.compat.v1.disable_eager_execution()

# 데이터셋 생성
texts = df[:10]['제목']

labels = [8,4,7,6,7,5,5,5,5,5]

# 어휘 사전의 크기가 200이라고 가정
vocab_size = 200

# 토큰화 및 정수 인코딩
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 패딩
max_len = max(len(seq) for seq in sequences)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len, padding='post')

# 모델 구성
num_blocks = 2
embed_dim = 32
num_heads = 4
ff_dim = 32

# 모델 생성
model = keras.Sequential([
    TransformerEncoder(num_blocks, embed_dim, num_heads, ff_dim, vocab_size),
    layers.GlobalAveragePooling1D(),
    layers.Dense(1, activation='linear')  # 이진 분류를 위한 출력 레이어
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
labels = np.array(labels)
model.fit(padded_sequences, labels, epochs=10, batch_size=2)

# 새로운 텍스트에 대한 예측
new_texts = [
    "삼성전자 이번 해 대박이다",
    "반도체 시장 완전 혹한기..."
]

new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=max_len, padding='post')

predictions = model.predict(new_padded_sequences)
print(predictions)


Train on 10 samples
Epoch 1/10
10/10 [==============================] - 1s 103ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 2/10
10/10 [==============================] - 0s 4ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 3/10
10/10 [==============================] - 0s 5ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 0s 5ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 5/10
10/10 [==============================] - 0s 5ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 6/10
10/10 [==============================] - 0s 5ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 7/10
10/10 [==============================] - 0s 4ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 8/10
10/10 [==============================] - 0s 4ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 9/10
10/10 [==============================] - 0s 5ms/sample - loss: -72.0662 - accuracy: 0.0000e+00
Epoch 10/10
10/10 [=====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[2.8446996]
 [2.9840188]]
